참고: https://github.com/malywut/gpt_examples/blob/main/Chap3_03_QuestionAnsweringOnPDF

In [21]:
# 라이브러리

import redis
from pypdf import PdfReader
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np

In [19]:
# 세팅

INDEX_NAME = "embeddings-index"
PREFIX = "doc"
REDIS_HOST = "localhost"
REDIS_PORT = 6379
DISTANCE_METRIC = "COSINE"
REDIS_PASSWORD = ""

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [22]:
# 레디스 클라이언트 초기화

class DataService():
    def __init__(self):
        self.redis_client = redis.Redis(
            host=REDIS_HOST,
            port=REDIS_PORT,
            password=REDIS_PASSWORD,
        )
    
    # 임베딩을 레디스에 저장하는 함수
    def load_data_to_redis(self, embeddings):
        for embedding in embedding:
            key = f"{PREFIX}:{str(embedding['id'])}"
            embedding["vector"] = np.array(embedding["vector"], dtyp=np.float32).tobytes()
            self.redis_client.hset(key, mapping=embedding)

In [20]:
# PDF 파일에서 읽은 데이터로 임베딩을 생성하는 함수

def pdf_to_embeddings(self, pdf_path: str, chunk_length: int = 1000):
    # PDF 파일에서 데이터를 읽고 분할
    reader = PdfReader(pdf_path)
    chunks = []

    for page in reader.pages:
        text_page = page.extract_text()
        chunks.extend([text_page[i:i+chunk_length]
                       for i in range(0, len(text_page), chunk_length)])
    
    # 임베딩 생성
    response = client.embeddings.create(model="text-embedding-ada-002", input=chunks)
    return [{'id': value['index'],
             'vector': value['embedding'],
             'text': chunks[value['index']]} for value in response['choices']]

In [1]:
import redis

conn = redis.StrictRedis(host="127.0.0.1", port=6379, db=0)
conn.set("test", "hello, world")

True

redis 실행 명령어: redis-cli -p 6379

docker 설치
https://velog.io/@gmlstjq123/Docker-%EC%84%A4%EC%B9%98

docker에 redis 설치하고 실행해서 연결하기(FT 명령어 쓰려면 도커 사용해야 함)
https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/docker/

나중에 streamlit 배포할 때 참고할 문서
https://towardsai.net/p/machine-learning/a-step-by-step-guide-to-developing-a-streamlit-application-with-redis-data-storage-and-deploying-it-using-docker